In [10]:
# common
import os
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))

from datetime import datetime, timedelta
import json

from io import BytesIO
from pathlib import Path

# network & protocol
import requests
from requests.exceptions import BaseHTTPError

# computational
import numpy as np
import pandas as pd
from sympy import var, Eq, solve

In [2]:
start_date = '1990-01-01'
end_date = '2019-12-31'
full_dates = pd.date_range(datetime.strptime(start_date, '%Y-%m-%d'),
                           datetime.strptime(end_date, '%Y-%m-%d'), freq='d')\
             .strftime('%Y-%m-%d').tolist()

In [3]:
# stations with no discharge to calculate rating curves
# Kg. Chhnang, Kompong Luong,
stations = [
    {
        'name': 'Battambang',
        'fitting': 'exp',
    },
    {
        'name': 'Chaktomuk',
        'fitting': 'poly_3',
    },
    {
        'name': 'Kg._Thmar',
        'fitting': 'poly_3',
    },
    {
        'name': 'Koh_Khel',
        'fitting': 'poly_3',
    },
    {
        'name': 'Kompong_Cham',
        'fitting': 'poly_3',
    },
    {
        'name': 'Kompong_Chen',
        'fitting': 'poly_3',
    },
    {
        'name': 'Kompong_Kdei',
        'fitting': 'poly_3',
    },
    {
        'name': 'Kompong_Thom',
        'fitting': 'exp',
    },
    {
        'name': 'Kratie',
        'fitting': 'poly_3',
    },
    {
        'name': 'Lumphat',
        'fitting': 'poly_3',
    },
    {
        'name': 'Neak_Luong',
        'fitting': 'poly_3',
    },
    {
        'name': 'Phnom_Penh_Port',
        'fitting': 'poly_3',
    },
    {
        'name': 'Prek_Kdam',
        'fitting': 'poly_3',
    },
    {
        'name': 'Siempang',
        'fitting': 'poly_3',
    },
    {
        'name': 'Sisophon',
        'fitting': 'exp',
    },
    {
        'name': 'Stung_Treng',
        'fitting': 'poly_3',
    },
    {
        'name': 'Voeun_Sai',
        'fitting': 'linear',
    },
]

In [6]:
def open_load_json(file_path, eqn):
    f = open(file_path)
    parameters = json.load(f)
    f.close()
    if eqn == 'exp' or eqn == 'linear':
        return parameters['m'], parameters['c']
    elif eqn == 'poly_2':
        return parameters['a'], parameters['b'], parameters['c']
    elif eqn == 'poly_3':
        return parameters['w'], parameters['x'], parameters['y'], parameters['c']
    else:
        raise ValueError('only allowed fittings are exponential (as exp), 2nd order polynomial (as poly_2), and 3rd order polynomial (as poly_3)')


In [12]:
Path(f'{BASE_DIR}/mrc_observations').mkdir(parents=True, exist_ok=True)

no_fit_stations = ['Phnom_Penh_Port', 'Prek_Kdam']

for _station in stations:
    station = _station['name']
    if station not in no_fit_stations:
        print('-------------------------------------------')
        print(f'station: {station}')

        df_discharge = pd.read_csv(f'{BASE_DIR}/pre_rating_curves/{station}/discharge.csv', index_col='datetime')
        df_discharge.index = pd.to_datetime(df_discharge.index)

        df_stage = pd.read_csv(f'{BASE_DIR}/pre_rating_curves/{station}/stage.csv', index_col='datetime')
        df_stage.index = pd.to_datetime(df_stage.index)

        discharge_mask = (df_discharge.index >= start_date) & (df_discharge.index <= end_date)
        stage_mask = (df_stage.index >= start_date) & (df_stage.index <= end_date)

        df_discharge = df_discharge.loc[discharge_mask]
        df_stage = df_stage.loc[stage_mask]

        if len(df_discharge) > 0:
            dates = df_discharge.index.tolist()
            dates = [date.strftime('%Y-%m-%d') for date in dates]
        else:
            dates = []

        remaining_dates = [datetime.strptime(_date, '%Y-%m-%d').date() for _date in full_dates if _date not in dates]
        df_stage = df_stage.loc[(df_stage.index.isin(remaining_dates)), :]

        fit_eqn = _station['fitting']

        stage = df_stage.stage

        if fit_eqn == 'linear':
            m, c = open_load_json(f'{BASE_DIR}/rating_curves/{station}_parameters.json', fit_eqn)
            # exponential: Q = (stage - c) / m
            df_stage['discharge'] = (stage - c) / m
#             df_stage['discharge'] =  np.multiply(np.power(10, c), np.power(stage, m))
            print('linear')
            print(df_stage.loc[df_stage.discharge < 0])
        elif fit_eqn == 'exp':
            m, c = open_load_json(f'{BASE_DIR}/rating_curves/{station}_parameters.json', fit_eqn)
            # exponential: Q = 10 ^ ((log(stage) - c) / m)
            df_stage['discharge'] =  (10 ** (np.log10(df_stage.stage) - c) / m)
            print('exp')
            print(df_stage.loc[df_stage.discharge < 0])
        elif fit_eqn == 'poly_2':
            a, b, c = open_load_json(f'{BASE_DIR}/rating_curves/{station}_parameters.json', fit_eqn)
            # D = a*Q^2 + b*Q + c
            # yields Q^2 + (b/a) * Q + (c - D / a) = 0
            # of the form ax^2 + bx + c = 0 has solution
            # x = (-b (+-) sqrt(b^2 - 4ac)) / 2a
            coefficients = [1, (b/a), (c - stage)/a]
            solution = np.roots(coefficients)
            print(solution)
            print('poly_2')
#             print(df_stage.loc[df_stage.discharge < 0])
        elif fit_eqn == 'poly_3':
            w, x, y, c = open_load_json(f'{BASE_DIR}/rating_curves/{station}_parameters.json', fit_eqn)
            # Q = w*d^3 + x*d^2 + y*d + c
#             df_stage['discharge'] = np.multiply(w, np.power(stage, 3)) + np.multiply(x, np.power(stage, 2)) + np.multiply(y, stage) + c
            q = var('q')
            solution = solve(Eq(w*q**3 + x*q**2 + y*q + (c-stage), 0), q)
            print(solution)
            print('poly_3')
#             print(df_stage.loc[df_stage.discharge < 0])
        else:
            raise ValueError('only allowed fittings are exponential (as exp), 2nd order polynomial (as poly_2), and 3rd order polynomial (as poly_3)')

#         df_stage = df_stage.drop(['stage', 'manual'], axis = 1)
#         df_discharge = df_discharge.drop(['calculated'], axis=1)
        

#         df = pd.concat([df_stage, df_discharge])
#         df = df.sort_index()
#         df.to_csv(f'{BASE_DIR}/mrc_observations/{station}.csv', encoding='utf-8', index=True)

-------------------------------------------
station: Battambang
exp
Empty DataFrame
Columns: [stage, manual, discharge]
Index: []
-------------------------------------------
station: Chaktomuk


SympifyError: SympifyError: datetime
1990-03-23    5.16642095757842e-11*q**3 - 6.22430649563572e-...
1990-03-24    5.16642095757842e-11*q**3 - 6.22430649563572e-...
1990-03-25    5.16642095757842e-11*q**3 - 6.22430649563572e-...
1990-03-26    5.16642095757842e-11*q**3 - 6.22430649563572e-...
1990-03-27    5.16642095757842e-11*q**3 - 6.22430649563572e-...
                                    ...                        
2019-12-27    5.16642095757842e-11*q**3 - 6.22430649563572e-...
2019-12-28    5.16642095757842e-11*q**3 - 6.22430649563572e-...
2019-12-29    5.16642095757842e-11*q**3 - 6.22430649563572e-...
2019-12-30    5.16642095757842e-11*q**3 - 6.22430649563572e-...
2019-12-31    5.16642095757842e-11*q**3 - 6.22430649563572e-...
Name: stage, Length: 6754, dtype: object

In [25]:
len(full_dates)

10957

In [26]:
2098/len(full_dates)

0.19147576891484896